<a href="https://colab.research.google.com/github/ReynaQuita/NLP/blob/main/CPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -U tensorflow tensorflow-hub

In [2]:
# 依赖：
!pip install sentencepiece
!pip install jieba
!pip install regex
!pip install tensorflow
!pip install tensorflow-hub

     |████████████████████████████████| 1.2MB 4.0MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files

src = list(files.upload().values())[0]
# open('mylib.py','wb').write(src)
# import mylib

Saving gpt2_tokenizer.py to gpt2_tokenizer.py


In [4]:
import tensorflow_hub as hub
import tensorflow as tf

from gpt2_tokenizer import GPT2Tokenizer

In [ ]:
print(tf.__version__)

2.5.0


In [ ]:
print(hub.__version__)

0.12.0


In [5]:
tokenizer = GPT2Tokenizer(
    '/content/drive/MyDrive/Colab Notebooks/cpm-tf2/CPM-Generate/bpe_3w_new/vocab.json',
    '/content/drive/MyDrive/Colab Notebooks/cpm-tf2/CPM-Generate/bpe_3w_new/merges.txt',
    model_file='/content/drive/MyDrive/Colab Notebooks/cpm-tf2/CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [7]:
gpt = hub.load('/content/drive/MyDrive/Colab Notebooks/cpm-tf2/cpm-lm-tf2_v2/')

In [8]:
def sample(tokenizer, gpt, sentence, number=1, length=20, top_p=0.9, temperature=0.9):
    """
    numbert: 输出句子个数 (number of output sentence)
    length: 输出最大长度
    top_p: token的概率排在这以上才有效
    temperature: 温度
    """
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    ret = gpt.signatures['serving_default'](
        inp=inputs,
        length=length,
        top_p=tf.constant(top_p, tf.float32),
        temperature=tf.constant(temperature, tf.float32)
    )['output_0']
    return [
        tokenizer.decode(s).replace(' ', '')
        for s in ret.numpy()
    ]

# 问答例子 (Question-Answering)

In [9]:
def ask_gpt(question):
    q = f'''
问题：中国首都是哪里？
答案：北京
问题：美国的首都是哪里？
答案：华盛顿
问题：李白在哪个朝代？
答案：唐朝
问题：{question}
答案：'''
    ret = sample(tokenizer, gpt, q, 3, 10, top_p=0.9, temperature=0.9)
    answers = []
    for x in ret:
        a = x[len(q):]
        a = a.split('\n')[0]
        answers.append(a)
    return answers

In [10]:
ask_gpt('唐伯虎是哪个朝代的人？')

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.791 seconds.
Prefix dict has been built successfully.


['清朝', '明朝', '明朝']

In [11]:
ask_gpt('世界上最帅的是谁？')

['头文字D中的藤原拓海', '普京', '普京']

In [12]:
ask_gpt('世界上最聪明的人是谁？')

['尼古拉·特斯拉', '爱迪生', '比尔盖茨']

In [13]:
ask_gpt('李清照是哪个朝代的人？')

['宋代', '宋代', '北宋']

In [14]:
ask_gpt('关公和秦琼谁厉害？')

['关羽和张飞谁厉害?', '两个都是神人', '关公']

In [15]:
ask_gpt('如何赚一百万美元？')

['卖菜', '炒股', '卖菜']

In [17]:
ask_gpt('烧烤和火锅哪个好吃？')

['烤肉', '烤羊肉串', '不知道']

In [18]:
ask_gpt('如何成为老师？')

['老师是一种职业', '老师是什么?', '认真读书,听老师的话']

In [19]:
ask_gpt('佩奇是什么动物？')

['浣熊', '浣熊', '鳄鱼']

In [20]:
ask_gpt('大象有几条腿？')

['象腿', '八条腿', '它是两条腿']

In [21]:
ask_gpt('姚明有多高？')

['203cm', '1米72', '1米74']

In [22]:
ask_gpt('汤姆和杰瑞谁厉害？')

['汤姆', '杰瑞', 'Tom和Jerry']

In [23]:
ask_gpt('野原新之助能打过樱桃子吗？')

['没有', '能', '不知道']

In [24]:
ask_gpt('学音乐和学画画哪个好？')

['两者都很好', '都好', '都不好']

In [25]:
ask_gpt('中文和英文哪个难学？')

['中文', '英文', '中文']

In [26]:
ask_gpt('python和java哪个难？')

['Java', 'c&java', 'Java']

In [27]:
ask_gpt('“锄禾日当午”的下一句是什么？')

['大年初一', '“汗滴禾下土”', '“汗滴禾下土”']

In [28]:
ask_gpt('烤鸭的主要原料是什么？')

['鸭肉', '鸭子', '鸭']

In [29]:
ask_gpt('把大象放冰箱总共分几步？')

['第一步:首先,放进', '第一步:先把大象放进冰箱,', '第一步:打开冰箱门,把大象']

In [30]:
ask_gpt('珠穆朗玛峰有多高？')

['1088米', '8848米', '4200米']

In [31]:
ask_gpt('世界上最高的山是什么？')

['珠穆朗玛峰', '珠穆朗玛峰', '珠穆朗玛峰']

In [32]:
ask_gpt('中国有多少人？')

['中国有7500万', '一千多万', '100万人']

In [33]:
ask_gpt('日本有多少人？')

['249万', '409', '只有两万人']

In [34]:
ask_gpt('苹果手机好还是华为手机好？')

['苹果', '苹果', '“你懂的”']

# 推理 (Reasoning)

In [35]:
ret = sample(tokenizer, gpt, '小明决定去吃饭，小红继续写作业\n问题：去吃饭的人是谁？\n答案：', 3, 10, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小红。
解析:第4题
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:我
小红:看!
小明
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小明,小红
【精彩片段
--------------------


# 英文问答例子 (English Entity Generation)

In [36]:
ret = sample(tokenizer, gpt, '默写英文：\n狗dog\n猫cat\n鸟', 3, 10, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写英文:
狗dog
猫cat
鸟bird
猫头鹰crow
--------------------
默写英文:
狗dog
猫cat
鸟bird
鱼tao
飞
--------------------
默写英文:
狗dog
猫cat
鸟bird
洋娃娃ballet
--------------------


# 默写古诗例子 (Chinese Poems LM)

In [37]:
ret = sample(tokenizer, gpt, '默写古诗：\n白日依山尽，黄河入海流。\n床前明月光，', 3, 10, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望
--------------------


# 不同的角色对话生成例子 (Dialogue Generation)

In [38]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n佟掌柜：“', 3, 20, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
佟掌柜:“这位少爷,你也是个买卖人,得给咱几个
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“刘麻子,你也该出来了,这破地方我也受够了
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“怎么,还有脸回?”
燕小六:“
--------------------


In [39]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n白展堂：“', 3, 20, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
白展堂:“白大哥,你要跑就走吧,我不拦你。”
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“你这是怎么了?”
包不同:“各回各家
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“你们以后不要再见面了,这件事我真的帮不了你。”
--------------------


In [40]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n莫小贝：“', 3, 20, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
莫小贝:“我等了你六年!”
韦小宝:“等了
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“我跟你有仇,我⁇你妈。”
莫
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“师父,我做梦也没想到,有人会那么对我。
--------------------


In [41]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n李白：“', 3, 20, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
李白:“你们几个小年轻儿,不懂事,一窝蜂的上,上
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“我去也!”
黄巢:“不想干就滚
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“金鳞岂是池中物,一遇风云便化龙。”
--------------------


# 问答的例子 (Entity Generation)

In [42]:
ret = sample(tokenizer, gpt, '中国的首都是北京\n日本的首都是东京\n美国的首都是', 3, 3, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
法国
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
韩国
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
世界
--------------------


In [43]:
ret = sample(tokenizer, gpt, '李白所在朝代是唐\n李清照所在朝代是宋\n唐伯虎所在朝代是', 3, 1, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明
--------------------


In [44]:
ret = sample(tokenizer, gpt, '陈奕迅是歌星\n郭德纲是', 3, 1, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

陈奕迅是歌星
郭德纲是评书
--------------------
陈奕迅是歌星
郭德纲是相声演员
--------------------
陈奕迅是歌星
郭德纲是相声
--------------------


In [45]:
ret = sample(tokenizer, gpt, '珠穆朗玛峰的高度（米）是', 3, 10, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

珠穆朗玛峰的高度(米)是13848,而南美洲最高峰—
--------------------
珠穆朗玛峰的高度(米)是在未开发前3000米(-5000米
--------------------
珠穆朗玛峰的高度(米)是2346米,仅次于珠穆朗玛
--------------------


In [46]:
ret = sample(tokenizer, gpt, '姚明的身高（米）是', 3, 10, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

姚明的身高(米)是六英尺三英寸,邓肯的身高
--------------------
姚明的身高(米)是1.75米,
--------------------
姚明的身高(米)是多少?今天看到福原爱在
--------------------


# 算数例子 (Arithmatic)

In [47]:
ret = sample(tokenizer, gpt, '1+1=2\n2+2=4\n3+3=6\n4+4=', 3, 1, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
2+2=4
3+3=6
4+4=12
--------------------
1+1=2
2+2=4
3+3=6
4+4=8
--------------------
1+1=2
2+2=4
3+3=6
4+4=12
--------------------


In [48]:
ret = sample(tokenizer, gpt, '1+1=2\n1+2=3\n1+3=4\n1+4=', 3, 1, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
1+2=3
1+3=4
1+4=5
--------------------
1+1=2
1+2=3
1+3=4
1+4=5
--------------------
1+1=2
1+2=3
1+3=4
1+4=5
--------------------


# ???的例子 (Language Modeling)

In [49]:
ret = sample(tokenizer, gpt, '''惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
''', 3, 30, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
紫金锤火紫金闪
八方云动山河动
青天白日紫天雷
赤云红光九龙
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
天雷这九天雷劫
烈火这九天大劫
金光这九天金光
金木水火土
神雷这
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
玄元这玄真金
金水这玄真木
木火两相生
战气磅礴灭天
--------------------


# 写作文例子 (LM)

In [50]:
ret = sample(tokenizer, gpt, '''爱情是''', 3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱情是怎么开始的。男:1、在恋爱中会经常说一些类似于“我对你的爱很深”“你是我的女神”“我对你的爱
--------------------
爱情是不一样的,我怎么确定对方是不是也是这样想的呢?万一错过了对的人,年纪大了怎么办?家里人知道了肯定会难过,自己也会愧疚一辈子。
--------------------
爱情是一场持久战,男女双方一定要保持自己的姿态,不要被一些不良的想法干扰,合理保持理性,建立一种有你在的爱的气氛,这点很重要。4
--------------------


In [52]:
ret = sample(
    tokenizer, gpt,
    '''一时黛玉进了荣府，下了车。众嬷嬷引着，便往东转弯，穿过一个东西的穿堂，向南大厅之后，仪门内大院落，上面五间大正房，两边厢房鹿顶耳房钻山，四通八达，轩昂壮丽，比贾母处不同。黛玉便知这方是正经正内室，一条大甬路，直接出大门的。''',
    3, 200, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。一路向南,至内院花园,穿过两边高墙,便到后院了。只见一簇柏树,大约八九十棵,都是三四百年以上的古柏,槎丫交柯,郁郁苍苍,不⁇壮观。又看过东南屋后,有一座园子,大约有一亩空地,四面俱是树木,中间是个长方池子,四周都是花圃,种着奇花异草,也有桂花、海棠、苹果、杏子、樱桃之类,上面架着数架白玉栏杆,中间是个夹道的池塘,水边有一座石桥,池中浮着数尾鲤鱼,——“泉声”。想必是
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。宝玉又想,这里又是什么正宫娘娘所居?前日说送燕窝来,怎么现在竟变成了燕窝铺,只是又没有做得巧妙了。正想着,只见黛玉出来,宝钗忙上来问道:“姐姐往那里去?”黛玉道:“往北方,再过不曾走过。”宝钗道:“这个北,还有个兜儿?”黛玉道:“有的。”宝钗就说:“这个兜子,里面放着的是书,你都看过了,还有什么念头没有?”黛玉道:“都烧了。”宝钗道:“有工夫看这个,还不如
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。大院落内外,两旁种着苍松翠柏,中间夹着块五色琉璃砖,砌成粉墙,围住着内室。黛玉见了,心中暗想:“这一间房倒象个花园,还要如何?”
--------------------


# 对话例子 (Dialogue Generation)

In [54]:
ret = sample(
    tokenizer, gpt,
    '''A：“今天我想吃火锅”
B：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“今天我想吃火锅”
B:“行,你想吃什么”
A:“麻辣烫”
B:“我知道了,我要吃火锅”
A:“嗯,
--------------------
A:“今天我想吃火锅”
B:“火锅?我就想吃火锅,你想吃火锅我就请客”
A:“好,”
B:“A你请客”
A:“我不
--------------------
A:“今天我想吃火锅”
B:“不好意思,今天我想吃小龙虾”
C:“今天我想吃烧烤”
D:“不好意思,今天我想吃螃蟹”
这种对话经常
--------------------


In [55]:
ret = sample(
    tokenizer, gpt, '''A：“跟我一起去看电影吧”
B：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“跟我一起去看电影吧”
B:“好啊”
C:“我可以坐你旁边吗?”
D:“可以啊”
E:“我看过了,觉得很无聊”

--------------------
A:“跟我一起去看电影吧”
B:“我没时间”
C:“那你呢?”
A:“我...我......”
C:“你也没时间?”
A:
--------------------
A:“跟我一起去看电影吧”
B:“那我就得待在家里,我没车”
A:“别啊,我带你去看电影吧”
B:“那我待在家里又不舒服
--------------------


# 对联例子 (LM)

In [56]:
duilian = '''
上联天，下联地
上联雨，下联风
上联大陆，下联长空
上联雷隐隐，下联雾蒙蒙
上联开心大吉，下联万事亨通
'''

In [57]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联李白作诗，下联',
    5, 2, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫作诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联王安石作
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联诗仙李白
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫作诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫作诗
--------------------


In [58]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联追求真爱，下联',
    5, 4, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联组建家庭

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联诚心交友
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联追逐金钱
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联吉祥如意

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联事业成功
上
--------------------


In [59]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天天向上，下联',
    5, 4, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联万事如意

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联平平安安

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联真火炎上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联快乐无边
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联大团圆
上
--------------------


In [60]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天地在我心，下联',
    5, 4, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联鬼神在我梦
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联礼义廉耻
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联春暧花开
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联鬼神在我胸
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联情满天下

--------------------


# 名人名言 (LM-Chinese Idiom?)

In [61]:
ret = sample(
    tokenizer, gpt, '''鲁迅曾经说过：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

鲁迅曾经说过:“今日之事,不单是文字狱,实系今世修《古文观止》之人,逐字逐句逐段逐段追查《论语》中‘君子’‘
--------------------
鲁迅曾经说过:“女人是一种很可怜的动物,男人若是欺负她,她就会跟着男人走。”然而,另一位中国女作家则认为:“男人们之
--------------------
鲁迅曾经说过:“要把小说当作历史来读,不能抱着猎奇的态度去读。”鲁迅的这个看法在今天看来有些偏激,因为小说是在真实的历史里发生的
--------------------


In [62]:
ret = sample(
    tokenizer, gpt, '''爱因斯坦曾经说过：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱因斯坦曾经说过:“所谓的物理学不过是使我们能够重复已知的事情罢了。”
--------------------
爱因斯坦曾经说过:“所有的问题都是一个道理:‘为什么’是不可以问的。”但是,他并没有说“不为什么”。他认为
--------------------
爱因斯坦曾经说过:“只有神才能真正改变物质世界,因为只有神才知道为什么物质世界会是这个样子。”9月9日,在陈云报告的基础上,中央决定
--------------------


In [63]:
ret = sample(
    tokenizer, gpt, '''牛顿曾经说过：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

牛顿曾经说过:“如果他喜欢的是布施,而不是别的东西,那么布施的结果也就会是宽恕。”对富人来说,钱并非一种选择,它只是一
--------------------
牛顿曾经说过:“我宁愿做快乐的傻子,也不愿做傻子式的人。”如果你按照这个标准选择职业,我不知道你还能干什么。我想
--------------------
牛顿曾经说过:“自从世界上出现了蒸汽机和电报,有些人就认为人是无所不能的。”在蒸汽机出现之前,甚至在人们开始认识到电和磁之前,
--------------------


In [64]:
ret = sample(
    tokenizer, gpt, '''佛祖曾经说过：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

佛祖曾经说过:“一个人,不管什么时候开始修行,最重要的就是心。”所谓心,就是从悟到的那一刻开始,经过无数次的“观照”,逐
--------------------
佛祖曾经说过:“一沙一世界,一花一天堂,一叶一如来,一砂一极乐。”他和天,在佛教中,成了对世界的极致描述。如此
--------------------
佛祖曾经说过:“在众生面前,他总是赤裸裸的。”他对佛祖也同样心知肚明。在一个没有佛祖的时代,当一个人看到一个乞丐,会
--------------------


In [65]:
ret = sample(
    tokenizer, gpt, '''乔布斯曾经说过：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

乔布斯曾经说过:“我们的大脑对输入的东西会进行预处理,让它们变成更加有意思的东西。”在《大教堂与集市》一书中,作者法比奥·
--------------------
乔布斯曾经说过:“如果你看到一个漂亮的女生,你只需做一件事:就是在她身上花足够的时间。”这句话说的就是这个道理。
--------------------
乔布斯曾经说过:“软件设计是为人服务的,如果我们要决定一个软件是用来解决什么问题的,最好是问使用者。”一个成功的软件设计应该是以人为本的,而不仅仅
--------------------
